In [ ]:
import pandas as pd

df = pd.read_csv("data/telco_churn.csv")
df.head()


In [ ]:
df.columns = (
    df.columns
    .str.strip()
    .str.lower()
    .str.replace(" ", "_")
)

df.columns


In [ ]:
df["lifecycle_stage"] = pd.cut(
    df["tenure_months"],
    bins=[-1, 3, 12, df["tenure_months"].max()],
    labels=["onboarding", "early_lifecycle", "established"]
)

df[["tenure_months", "lifecycle_stage"]].head(10)


In [ ]:
churn_by_lifecycle = (
    df.groupby("lifecycle_stage")["churn_label"]
      .apply(lambda x: (x == "Yes").mean() * 100)
      .round(2)
      .reset_index(name="churn_rate_pct")
)

churn_by_lifecycle


In [ ]:
contract_lifecycle_churn = (
    df.groupby(["lifecycle_stage", "contract"], observed=False)["churn_label"]
      .apply(lambda x: (x == "Yes").mean() * 100)
      .round(2)
      .reset_index(name="churn_rate_pct")
)

contract_lifecycle_churn.sort_values("churn_rate_pct", ascending=False)


In [ ]:
import sqlite3

conn = sqlite3.connect("crm_churn.db")
df.to_sql("telco_customers", conn, if_exists="replace", index=False)
conn.close()


In [ ]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("crm_churn.db")

query = """
SELECT
  lifecycle_stage,
  ROUND(
    100.0 * SUM(CASE WHEN churn_label = 'Yes' THEN 1 ELSE 0 END) / COUNT(*),
    2
  ) AS churn_rate_pct
FROM telco_customers
GROUP BY lifecycle_stage
ORDER BY churn_rate_pct DESC;
"""

pd.read_sql(query, conn)


In [ ]:
query = """
SELECT
  lifecycle_stage,
  contract,
  ROUND(
    100.0 * SUM(CASE WHEN churn_label = 'Yes' THEN 1 ELSE 0 END) / COUNT(*),
    2
  ) AS churn_rate_pct
FROM telco_customers
GROUP BY lifecycle_stage, contract
ORDER BY churn_rate_pct DESC;
"""

pd.read_sql(query, conn)


In [ ]:
conn.close()
